In [ ]:
#Kết nối GG-Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/Shareddrives/Dehusked_Rice
!python -m pip install pyyaml
import sys, os, distutils.core
#!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

/content/drive/Shareddrives/Dehusked_Rice
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 18.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61400 sha256=9506cdfa99baed916d7e1e7e52ff50e0514f99554778d9b120616e1e340545f5
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144554 sha256=0a72a1f926a1dc45d5bd3102a009c0b7b38b450b98c60d11d41de5a4a3b3d108
  Stored in direct

In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
torch:  2.2 ; cuda:  cu121
detectron2: 0.6


In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from matplotlib import pyplot as plt
from detectron2.engine import DefaultTrainer
import yaml
from detectron2.utils.visualizer import ColorMode
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

In [ ]:
register_coco_instances("my_dataset_train",{},"/content/drive/Shareddrives/Dehusked_Rice/DehuskedRice/final_data.coco/train/_annotations.coco.json","/content/drive/Shareddrives/Dehusked_Rice/DehuskedRice/final_data.coco/train")
train_metadata = MetadataCatalog.get("my_dataset_train")
train_dataset_dicts = DatasetCatalog.get("my_dataset_train")

[05/12 02:26:08 d2.data.datasets.coco]: Loading /content/drive/Shareddrives/Dehusked_Rice/DehuskedRice/final_data.coco/train/_annotations.coco.json takes 4.69 seconds.
WARNING [05/12 02:26:08 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[05/12 02:26:08 d2.data.datasets.coco]: Loaded 270 images in COCO format from /content/drive/Shareddrives/Dehusked_Rice/DehuskedRice/final_data.coco/train/_annotations.coco.json


In [ ]:
%cd /content/drive/Shareddrives/Dehusked_Rice/Detectron2_Model
output_dir='/content/drive/Shareddrives/Dehusked_Rice/Detectron2_Model/OUTPUT'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
cfg = get_cfg()
cfg.OUTPUT_DIR = output_dir
cfg.merge_from_file("/content/drive/Shareddrives/Dehusked_Rice/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2  # This is the real "batch size" commonly known to deep learning people
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 10000   # 1000 iterations seems good enough for this dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   # Default is 512, using 256 for this dataset.
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) #Create an instance of of DefaultTrainer with the given congiguration
trainer.resume_or_load(resume=False)

/content/drive/Shareddrives/Dehusked_Rice/Detectron2
[05/10 06:41:45 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num

model_final_f10217.pkl: 178MB [00:00, 264MB/s]                           
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
roi_heads.mask_head.predictor.{bias, weight}


In [ ]:
trainer.train()

[05/10 06:43:17 d2.engine.train_loop]: Starting training from iteration 0


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[05/10 06:43:41 d2.utils.events]:  eta: 2:13:26  iter: 19  total_loss: 3.854  loss_cls: 1.582  loss_box_reg: 0.6918  loss_mask: 0.6948  loss_rpn_cls: 0.857  loss_rpn_loc: 0.06368    time: 0.7722  last_time: 0.8033  data_time: 0.5391  last_data_time: 0.3428   lr: 4.9953e-06  max_mem: 2100M
[05/10 06:43:59 d2.utils.events]:  eta: 2:14:07  iter: 39  total_loss: 3.079  loss_cls: 1.447  loss_box_reg: 0.7503  loss_mask: 0.6776  loss_rpn_cls: 0.1692  loss_rpn_loc: 0.05972    time: 0.7726  last_time: 0.9299  data_time: 0.2953  last_data_time: 0.3939   lr: 9.9902e-06  max_mem: 2101M
[05/10 06:44:17 d2.utils.events]:  eta: 2:15:30  iter: 59  total_loss: 2.7  loss_cls: 1.222  loss_box_reg: 0.7536  loss_mask: 0.6425  loss_rpn_cls: 0.02222  loss_rpn_loc: 0.06027    time: 0.8017  last_time: 1.0447  data_time: 0.3754  last_data_time: 0.5285   lr: 1.4985e-05  max_mem: 2101M
[05/10 06:44:33 d2.utils.events]:  eta: 2:14:46  iter: 79  total_loss: 2.366  loss_cls: 0.9517  loss_box_reg: 0.7469  loss_mask: 

In [ ]:
# Save the configuration to a config.yaml file
# Save the configuration to a config.yaml file
config_dir='/content/drive/Shareddrives/Dehusked_Rice/Detectron2_Model/Detectron2_Models'
if not os.path.exists(config_dir):
    os.makedirs(config_dir)
config_yaml_path = "/content/drive/Shareddrives/Dehusked_Rice/Detectron2_Model/Detectron2_Models/config.yaml"
with open(config_yaml_path, 'w') as file:
    yaml.dump(cfg, file)

### Final Train

In [ ]:
%cd /content/drive/Shareddrives/Dehusked_Rice/Detectron2_Model
output_dir='/content/drive/Shareddrives/Dehusked_Rice/Detectron2_Model/OUTPUT_FINAL'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
cfg = get_cfg()
cfg.OUTPUT_DIR = output_dir
cfg.merge_from_file("/content/drive/Shareddrives/Dehusked_Rice/Detectron2_Model/Detectron2_Models/config.yaml")
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = "/content/drive/Shareddrives/Dehusked_Rice/Detectron2_Model/OUTPUT/model_final.pth" # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2  # This is the real "batch size" commonly known to deep learning people
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 10000   # 1000 iterations seems good enough for this dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   # Default is 512, using 256 for this dataset.
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) #Create an instance of of DefaultTrainer with the given congiguration
trainer.resume_or_load(resume=False)

/content/drive/Shareddrives/Dehusked_Rice/Detectron2


[05/12 02:28:16 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

In [ ]:
trainer.train()

[05/12 02:28:59 d2.engine.train_loop]: Starting training from iteration 0


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[05/12 02:29:17 d2.utils.events]:  eta: 1:21:27  iter: 19  total_loss: 0.1871  loss_cls: 0.04045  loss_box_reg: 0.07388  loss_mask: 0.04431  loss_rpn_cls: 0.0001791  loss_rpn_loc: 0.02334    time: 0.4977  last_time: 0.5050  data_time: 0.2026  last_data_time: 0.0263   lr: 4.9953e-06  max_mem: 2100M
[05/12 02:29:32 d2.utils.events]:  eta: 1:19:58  iter: 39  total_loss: 0.1966  loss_cls: 0.0505  loss_box_reg: 0.07578  loss_mask: 0.04485  loss_rpn_cls: 0.0002602  loss_rpn_loc: 0.02559    time: 0.4972  last_time: 0.7271  data_time: 0.0341  last_data_time: 0.2401   lr: 9.9902e-06  max_mem: 2101M
[05/12 02:29:43 d2.utils.events]:  eta: 1:22:55  iter: 59  total_loss: 0.1894  loss_cls: 0.03949  loss_box_reg: 0.07533  loss_mask: 0.04724  loss_rpn_cls: 0.0002277  loss_rpn_loc: 0.02556    time: 0.5174  last_time: 0.6142  data_time: 0.0830  last_data_time: 0.1111   lr: 1.4985e-05  max_mem: 2101M
[05/12 02:29:53 d2.utils.events]:  eta: 1:22:51  iter: 79  total_loss: 0.1977  loss_cls: 0.04189  loss_b

In [ ]:
# Save the configuration to a config.yaml file
# Save the configuration to a config.yaml file
config_dir='/content/drive/Shareddrives/Dehusked_Rice/Detectron2_Model/Detectron2_Models_Final'
if not os.path.exists(config_dir):
    os.makedirs(config_dir)
config_yaml_path = "/content/drive/Shareddrives/Dehusked_Rice/Detectron2_Model/Detectron2_Models_Final/config.yaml"
with open(config_yaml_path, 'w') as file:
    yaml.dump(cfg, file)